# Introduction - Oracle Version 2

Notebook to load **bug reports**, **test cases** datasets and the **feature_matrixes** from the expert and volunteers responses given in the PyBossa applications, and create from them the **oracle** dataset. 

In this notebook we create a version of the oracle based on the results obtained from the empirical study made with volunteers and the PyBossa application. The relationship between bug reports and test cases is established through the firefox features shared between both artifacts.

Is expected that this oracle be more precise than the version created on the previous notebook (__oracle_v1__), once the trace links are created based on the existing relationship between the bug report and a given Firefox Feature.

# Load Libraries and Data

In [4]:
from mod_finder_util import mod_finder_util
mod_finder_util.add_modules_origin_search_path()

import pandas as pd
import numpy as np
from sklearn.externals.joblib import Parallel, delayed
from tqdm import tqdm

from modules.utils import aux_functions
from modules.utils import firefox_dataset_p2 as fd

In [5]:
testcases = fd.read_testcases_df()
bugreports = fd.read_bugreports_df()
features = fd.read_features_df()
expert_matrix = fd.read_expert_matrix_df()
volunteers_matrix = fd.read_volunteers_matrix_df()

TestCases.shape: (207, 12)
BugReports.shape: (93, 18)
Features.shape: (21, 8)
Expert Matrix shape: (93, 21)
Volunteers Matrix shape: (73, 21)


# Oracles

In this section we create both oracles Features x Bug Reports and Test Cases x Bug Reports oracles from the data obtained from the empirical study.

## Features x Bug Reports Trace Matrix

### Create Features x Bug Reports Trace Matrix

In [11]:
feat_br_matrix = pd.DataFrame(index=bugreports.br_name, 
                              columns=features.feat_name, 
                              data=np.zeros((len(bugreports),len(features)),dtype='int8'))

print('Features x Bug Reports Trace Matrix shape: {}'.format(feat_br_matrix.shape))

for idx,row in volunteers_matrix.iterrows():
    for col in volunteers_matrix.columns:
        br_name = 'BR_{}_SRC'.format(idx)
        if expert_matrix.at[idx,col] == volunteers_matrix.at[idx,col]:
            feat_br_matrix.at[br_name,col] = expert_matrix.at[idx,col]
        else:
            feat_br_matrix.at[br_name,col] = 0  

print(feat_br_matrix.shape)

Features x Bug Reports Trace Matrix shape: (93, 21)
(93, 21)


### Save Features x Bug Reports Trace Matrix

In [12]:
feat_br_matrix.T.to_csv('../data/mozilla_firefox_v2/firefoxDataset/oracle/output/firefox_v2/feat_br_matrix_emp_study.csv', index=True)

## Test Cases x Bug Reports Trace Matrix

### Get Features by Bug Report

In [20]:
def get_features(br_id, from_matrix):
    features_ids = ""
    for col in volunteers_matrix.columns:
        if from_matrix == "EXPERT_AND_VOLUNTEERS_MATRICES":
            if expert_matrix.at[br_id, col] == 1 and volunteers_matrix.at[br_id, col] == 1:
                if features_ids == "":
                    features_ids = str(expert_matrix.columns.get_loc(col) + 1)
                else:
                    features_ids = features_ids + " " + str(expert_matrix.columns.get_loc(col) + 1)
        elif from_matrix == "EXPERT_MATRIX":
            if expert_matrix.at[br_id, col] == 1:
                if features_ids == "":
                    features_ids = str(expert_matrix.columns.get_loc(col) + 1)
                else:
                    features_ids = features_ids + " " + str(expert_matrix.columns.get_loc(col) + 1)
        elif from_matrix == "VOLUNTEERS_MATRIX":
            if volunteers_matrix.at[br_id, col] == 1:
                if features_ids == "":
                    features_ids = str(volunteers_matrix.columns.get_loc(col) + 1)
                else:
                    features_ids = features_ids + " " + str(volunteers_matrix.columns.get_loc(col) + 1)
            
    return features_ids

matrices_names = [('exp_vol_m','EXPERT_AND_VOLUNTEERS_MATRICES'),
                  ('exp_m','EXPERT_MATRIX'),
                  ('vol_m','VOLUNTEERS_MATRIX')]

bugreports['Features_IDs_exp_vol_m'] = ""
bugreports['Features_IDs_exp_m'] = ""
bugreports['Features_IDs_vol_m'] = ""
for br_id in volunteers_matrix.index:
    for idx2, br in bugreports.iterrows():
        if br.Bug_Number == br_id:
            for mat_code,mat_name in matrices_names:
                bugreports.at[idx2, 'Features_IDs_'+mat_code] = get_features(br_id, from_matrix=mat_name)

bugreports[['Bug_Number','Features_IDs_exp_vol_m','Features_IDs_exp_m','Features_IDs_vol_m']].head()

,Bug_Number,Features_IDs_exp_vol_m,Features_IDs_exp_m,Features_IDs_vol_m
0,1181835,6,6,6
1,1248267,,,4
2,1248268,,,
3,1257087,1,1,1
4,1264988,,,


### Checking Link Condition Function

Check link condition function to decide if a given cell in the oracle has a positive link (1) or a negative link (0).

In [22]:
def check_link_condition(br, tc):
    if str(tc.Feature_ID) in br.Features_IDs_exp_vol_m.split(" "):
        return True
    return False

oracle_df = pd.DataFrame(columns=bugreports.br_name, index=testcases.tc_name, data=np.zeros(shape=(len(testcases),len(bugreports))), dtype='int8')
for idx_1,br in tqdm(bugreports.iterrows()):
    for idx_2,tc in testcases.iterrows():
        if check_link_condition(br, tc):
            oracle_df.at[tc.tc_name, br.br_name] = 1
        else:
            oracle_df.at[tc.tc_name, br.br_name] = 0


93it [00:02, 44.45it/s]


### Display Oracle

In [23]:
print('Oracle shape: {}\n'.format(oracle_df.shape))
display(oracle_df.head())

Oracle shape: (207, 93)



br_name,BR_1181835_SRC,BR_1248267_SRC,BR_1248268_SRC,BR_1257087_SRC,BR_1264988_SRC,BR_1267480_SRC,BR_1267501_SRC,BR_1269348_SRC,BR_1269485_SRC,BR_1270274_SRC,...,BR_1352539_SRC,BR_1353831_SRC,BR_1357085_SRC,BR_1357458_SRC,BR_1365887_SRC,BR_1408361_SRC,BR_1430603_SRC,BR_1432915_SRC,BR_1449700_SRC,BR_1451475_SRC
tc_name,,,,,,,,,,,,,,,,,,,,,
TC_1_TRG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TC_2_TRG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TC_3_TRG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TC_4_TRG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TC_5_TRG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Save Oracle

New Oracle Dataset
Dimension: 207 x 93

In [24]:
oracle_df.to_csv('../data/mozilla_firefox_v2/firefoxDataset/oracle/output/trace_matrix_final_emp_study.csv')

# Analysis

## Cohen's Kappa - Features x Bug Reports Trace Matrix

The Cohen's Kappa Score measures the iter-rater agreement for qualitive answers (categorical items). It is generally thought to be a more robust measure than simple percent agreement calculation, as κ takes into account the possibility of the agreement occurring by chance.

In the section bellow, we calculate the Cohen's kappa with the expert answers and the volunteers answers.

In [49]:
from sklearn.metrics import cohen_kappa_score

expert_answers = []
volunteers_answers = []

for idx,row in volunteers_matrix.iterrows():
    for col in volunteers_matrix.columns:
        volunteers_answers.append(volunteers_matrix.at[idx,col])
        expert_answers.append(expert_matrix.at[idx,col])

print("Expert Answers Length: {}".format(len(expert_answers)))
print("Volunteers Answers Length: {}".format(len(volunteers_answers)))

print("Cohen Kappa Score: {}".format(cohen_kappa_score(expert_answers, volunteers_answers)))

Expert Answers Length: 1533
Volunteers Answers Length: 1533
Cohen Kappa Score: 0.5273637418508574


## Cohen's Kappa - Test Cases x Bug Reports Trace Matrix

In the section bellow, we calculate the Cohen's kappa based on two matrices:

* the matrix of Test Cases x Bug Reports generated from the answers of the **expert**
* the matrix of Test Cases x Bug Reports generated from the answers of the **volunteers**

In [ ]:
from sklearn.metrics import cohen_kappa_score

expert_answers = []
volunteers_answers = []

for idx,row in volunteers_matrix.iterrows():
    for col in volunteers_matrix.columns:
        volunteers_answers.append(volunteers_matrix.at[idx,col])
        expert_answers.append(expert_matrix.at[idx,col])

print("Expert Answers Length: {}".format(len(expert_answers)))
print("Volunteers Answers Length: {}".format(len(volunteers_answers)))

print("Cohen Kappa Score: {}".format(cohen_kappa_score(expert_answers, volunteers_answers)))

## Analysis of Amount of Affected Test Cases by Features

Analysis of the amount of test cases that are directly affected by a bug report after made the traceability between bug reports and features in the empirical study. We analyse the amount of affected test cases using three different matrices: expert answers only, volunteers answers only and the result of expert and volunteers answers.

In [30]:
for idx, row in bugreports.iterrows():
    amount_aff_tcs_exp, amount_aff_tcs_vol, amount_aff_tcs_exp_vol = 0,0,0  # amount of affected testcases of feature
    for f_id in row.Features_IDs_exp_m.split(" "):
        if f_id != "":
            amount_aff_tcs_exp = amount_aff_tcs_exp + len(testcases[testcases.Feature_ID == int(f_id)])
    for f_id in row.Features_IDs_vol_m.split(" "):
        if f_id != "":
            amount_aff_tcs_vol = amount_aff_tcs_vol + len(testcases[testcases.Feature_ID == int(f_id)])
    for f_id in row.Features_IDs_exp_vol_m.split(" "):
        if f_id != "":
            amount_aff_tcs_exp_vol = amount_aff_tcs_exp_vol + len(testcases[testcases.Feature_ID == int(f_id)])

    bugreports.at[idx, 'Amount_Aff_TCs_Exp'] = amount_aff_tcs_exp
    bugreports.at[idx, 'Amount_Aff_TCs_Vol'] = amount_aff_tcs_vol
    bugreports.at[idx, 'Amount_Aff_TCs_Exp_Vol'] = amount_aff_tcs_exp_vol

cols = ['br_name','Features_IDs_exp_m','Features_IDs_vol_m','Features_IDs_exp_vol_m','Amount_Aff_TCs_Exp','Amount_Aff_TCs_Vol', 'Amount_Aff_TCs_Exp_Vol']   
display(bugreports[cols].head())

,br_name,Features_IDs_exp_m,Features_IDs_vol_m,Features_IDs_exp_vol_m,Amount_Aff_TCs_Exp,Amount_Aff_TCs_Vol,Amount_Aff_TCs_Exp_Vol
0,BR_1181835_SRC,6,6,6,31.0,31.0,31.0
1,BR_1248267_SRC,,4,,0.0,6.0,0.0
2,BR_1248268_SRC,,,,0.0,0.0,0.0
3,BR_1257087_SRC,1,1,1,13.0,13.0,13.0
4,BR_1264988_SRC,,,,0.0,0.0,0.0


In [44]:
positives_amount_exp_only = bugreports.Amount_Aff_TCs_Exp.sum()
negatives_amount_exp_only = len(bugreports) * len(testcases) - positives_amount_exp_only
print("Total Amount of Expected Positive Links - Expert Only: {}".format(positives_amount_exp_only))
print("Total Amount of Expected Negative Links - Expert Only: {}".format(negatives_amount_exp_only))
print("Positives Percentual: {:2.2f}%".format(float(positives_amount_exp_only/(positives_amount_exp_only+negatives_amount_exp_only)*100)))
print("Negatives Percentual: {:2.2f}%".format(float(negatives_amount_exp_only/(positives_amount_exp_only+negatives_amount_exp_only)*100)))
print()

positives_amount_vol_only = bugreports.Amount_Aff_TCs_Vol.sum()
negatives_amount_vol_only = len(bugreports) * len(testcases) - positives_amount_vol_only
print("Total Amount of Expected Positive Links - Volunteers Only: {}".format(positives_amount_vol_only))
print("Total Amount of Expected Negative Links - Volunteers Only: {}".format(negatives_amount_vol_only))
print("Positives Percentual: {:2.2f}%".format(float(positives_amount_vol_only/(positives_amount_vol_only+negatives_amount_vol_only)*100)))
print("Negatives Percentual: {:2.2f}%".format(float(negatives_amount_vol_only/(positives_amount_vol_only+negatives_amount_vol_only)*100)))
print()

positives_amount_exp_vol = bugreports.Amount_Aff_TCs_Exp_Vol.sum()
negatives_amount_exp_vol = len(bugreports) * len(testcases) - positives_amount_exp_vol
print("Total Amount of Expected Positive Links - Expert & Volunteers: {}".format(positives_amount_exp_vol))
print("Total Amount of Expected Negative Links - Expert & Volunteers: {}".format(negatives_amount_exp_vol))
print("Positives Percentual: {:2.2f}%".format(float(positives_amount_exp_vol/(positives_amount_exp_vol+negatives_amount_exp_vol)*100)))
print("Negatives Percentual: {:2.2f}%".format(float(negatives_amount_exp_vol/(positives_amount_exp_vol+negatives_amount_exp_vol)*100)))

Total Amount of Expected Positive Links - Expert Only: 754.0
Total Amount of Expected Negative Links - Expert Only: 18497.0
Positives Percentual: 3.92%
Negatives Percentual: 96.08%

Total Amount of Expected Positive Links - Volunteers Only: 920.0
Total Amount of Expected Negative Links - Volunteers Only: 18331.0
Positives Percentual: 4.78%
Negatives Percentual: 95.22%

Total Amount of Expected Positive Links - Expert & Volunteers: 512.0
Total Amount of Expected Negative Links - Expert & Volunteers: 18739.0
Positives Percentual: 2.66%
Negatives Percentual: 97.34%


## Analysis of No Maching Answers

In [47]:
cols = ['br_name','Features_IDs_exp_m','Features_IDs_vol_m','Features_IDs_exp_vol_m','Amount_Aff_TCs_Exp','Amount_Aff_TCs_Vol', 'Amount_Aff_TCs_Exp_Vol']
bugreports[bugreports.Features_IDs_exp_m != bugreports.Features_IDs_vol_m][cols].head(10)

,br_name,Features_IDs_exp_m,Features_IDs_vol_m,Features_IDs_exp_vol_m,Amount_Aff_TCs_Exp,Amount_Aff_TCs_Vol,Amount_Aff_TCs_Exp_Vol
1,BR_1248267_SRC,,4,,0.0,6.0,0.0
5,BR_1267480_SRC,3,,,22.0,0.0,0.0
7,BR_1269348_SRC,3,,,22.0,0.0,0.0
11,BR_1271607_SRC,1,4,,13.0,6.0,0.0
13,BR_1277937_SRC,,18 20,,0.0,21.0,0.0
14,BR_1278388_SRC,1,4 7,,13.0,12.0,0.0
24,BR_1287384_SRC,,3,,0.0,22.0,0.0
25,BR_1287687_SRC,4,3,,6.0,22.0,0.0
27,BR_1287823_SRC,1,4,,13.0,6.0,0.0
29,BR_1289832_SRC,1,18,,13.0,16.0,0.0
